In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 25 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

11 April 2023 ; 13:32:15


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,1,1
1,Bobby Portis,PF,27,MIL,70,22,26.0,5.7,11.5,0.496,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,1
4,Goran Dragić,PG,36,MIL,58,0,15.0,2.4,5.8,0.421,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,1
6,Jae Crowder,SF,32,MIL,18,3,18.9,2.5,5.2,0.479,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,1
7,Jevon Carter,PG,27,MIL,81,39,22.3,3.0,7.0,0.423,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,26.0,67,0,BOS,18.2,2.570149,7.098507,14.9,2,2.3,4.405970,0.318000
1,Immanuel Quickley,28.9,81,21,NYK,16.3,2.125926,6.782716,14.9,5,1.3,4.454321,0.132934
2,Bobby Portis,26.0,70,22,MIL,17.8,1.405714,6.208571,14.1,1,1.2,2.928571,0.104890
3,Bruce Brown,28.5,80,31,DEN,13.3,0.820000,4.407500,11.5,1,-1.1,1.742500,0.087863
4,Malik Monk,22.3,77,0,SAC,16.6,1.064935,3.407792,13.5,3,1.6,2.236364,0.076394


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)